In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import types as t
import math

In [4]:
spark = SparkSession \
    .builder \
    .appName("Data with Nikk the Greek Spark Session") \
    .master("local[4]") \
    .enableHiveSupport() \
    .config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.5") \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.0") \
    .getOrCreate()

"""
Reference gresearch:
- Parquet files analysis: https://www.gresearch.com/blog/article/parquet-files-know-your-scaling-limits/
- GitHub Spark extension: https://github.com/G-Research/spark-extension
- Parquet methods: https://github.com/G-Research/spark-extension/tree/master/python/gresearch/spark/parquet
"""

:: loading settings :: url = jar:file:/opt/homebrew/Caskroom/miniconda/base/envs/pyspark-2.5.0/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/SNV6FE/.ivy2/cache
The jars for the packages stored in: /Users/SNV6FE/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-82825e7b-7a82-4258-b012-49035b4ff7a0;1.0
	confs: [default]
You probably access the destination server through a proxy server that is not well configured.
You probably access the destination server through a proxy server that is not well configured.
You probably access the destination server through a proxy server that is not well configured.
You probably access the destination server through a proxy server that is not well configured.
:: resolution report :: resolve 100ms :: artifacts dl 0ms
	:: modules in use:
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	----------------------------------

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [3]:
sc = spark.sparkContext

In [4]:
#Turning off AQE as it generates more jobs which might be confusing for this scenario here. 
spark.conf.set("spark.sql.adaptive.enabled", "false")
#to not cache dataframes... this may not create repeatable results
spark.conf.set("spark.databricks.io.cache.enabled", "false")

In [5]:
def sdf_generator(num_rows: int, num_partitions: int = None) -> "DataFrame":
    return (
        spark.range(num_rows, numPartitions=num_partitions)
        .withColumn("date", f.current_date())
        .withColumn("timestamp",f.current_timestamp())
        .withColumn("idstring", f.col("id").cast("string"))
        .withColumn("idfirst", f.col("idstring").substr(0,1))
        .withColumn("idlast", f.col("idstring").substr(-1,1))
        )

In [6]:
sdf = sdf_generator(10000000, 8)
sdf.show()

+---+----------+--------------------+--------+-------+------+
| id|      date|           timestamp|idstring|idfirst|idlast|
+---+----------+--------------------+--------+-------+------+
|  0|2024-03-12|2024-03-12 06:58:...|       0|      0|     0|
|  1|2024-03-12|2024-03-12 06:58:...|       1|      1|     1|
|  2|2024-03-12|2024-03-12 06:58:...|       2|      2|     2|
|  3|2024-03-12|2024-03-12 06:58:...|       3|      3|     3|
|  4|2024-03-12|2024-03-12 06:58:...|       4|      4|     4|
|  5|2024-03-12|2024-03-12 06:58:...|       5|      5|     5|
|  6|2024-03-12|2024-03-12 06:58:...|       6|      6|     6|
|  7|2024-03-12|2024-03-12 06:58:...|       7|      7|     7|
|  8|2024-03-12|2024-03-12 06:58:...|       8|      8|     8|
|  9|2024-03-12|2024-03-12 06:58:...|       9|      9|     9|
| 10|2024-03-12|2024-03-12 06:58:...|      10|      1|     0|
| 11|2024-03-12|2024-03-12 06:58:...|      11|      1|     1|
| 12|2024-03-12|2024-03-12 06:58:...|      12|      1|     2|
| 13|202

# 1 - JSON

## 1-1 Initial state JSON
- Writing and reading json we default settings

Results:
- Write time: 8.4 s 
- Load time: 11.7 s
- Data size: 1208 MB

How it works: https://github.com/jerryshao/apache-spark/blob/master/sql/core/src/main/scala/org/apache/spark/sql/DataFrameReader.scala#L323
https://spark.apache.org/docs/latest/sql-data-sources-json.html

In [7]:
sc.setJobDescription("Save Json")
path_json = "D:/Spark/Data/format_json_large.json"
sdf.write.format("json").mode("overwrite").save(path_json)

In [11]:
#set maxPartitions MB for a fair comparison of 8 partitions also during read 
maxPartitionsMB = 160
maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")

In [12]:
sc.setJobDescription("Load Json without schema")
sdf_json = spark.read.format("json").load(path_json)
sdf_json.write.format("noop").mode("overwrite").save()

## 1-2 Column filter
- Use the schema of the loaded JSON Dataframe for loading the data and move to correct order

Results:
- Load time: 7 s

In [14]:
sc.setJobDescription("Parquet Column Filter")
sdf_json = spark.read.format("parquet").load(path_json)
sdf_json = sdf_json.select("id", "idstring")
sdf_json.write.format("noop").mode("overwrite").save()

# 2 - CSV with schema interference

## 2-1 Initial state CSV

- Writing and reading CSV we default settings

Results:
- Write time: 6.2 s
- Load time: 25.2 s
- Data size: 593 MB

How it works: https://github.com/jerryshao/apache-spark/blob/master/sql/core/src/main/scala/org/apache/spark/sql/DataFrameReader.scala#L470
https://spark.apache.org/docs/latest/sql-data-sources-csv.html

In [22]:
sc.setJobDescription("Save CSV")
path_csv = "D:/Spark/Data/format_csv_large.csv"
sdf.write.format("csv").mode("overwrite").option("header", "True").save(path_csv)

In [23]:
#set maxPartitions MB for a fair comparison of 8 partitions also during read 
maxPartitionsMB = 80
maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")

In [24]:
sc.setJobDescription("Load CSV without schema")
sdf_csv = spark.read.format("csv").options(inferSchema=True, header=True).load(path_csv)
sdf_csv.write.format("noop").mode("overwrite").save()

## 2-2 Column Filter
- Use the schema of the loaded JSON Dataframe for loading the data and move to correct order

Results:
- Load time: 11.5 s

In [26]:
sc.setJobDescription("Parquet Column Filter")
sdf_csv = spark.read.format("parquet").load(path_csv)
sdf_csv = sdf_csv.select("id", "idstring")
sdf_csv.write.format("noop").mode("overwrite").save()

## 2-3 Row Filter

# 3 - Parquet

## 3-1 Initial state Parquet
- Write time: 3.6 s
- Load time: 1 s
- Data size: 81.5 MB

How it works: https://spark.apache.org/docs/latest/sql-data-sources-parquet.html

In [34]:
sc.setJobDescription("Save Parquet")
path_parquet = "D:/Spark/Data/format_parquet_large.parquet"
sdf.write.format("parquet").mode("overwrite").save(path_parquet)

In [35]:
#set maxPartitions MB for a fair comparison of 8 partitions also during read 
maxPartitionsMB = 15
maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")

In [36]:
sc.setJobDescription("Load Parquet all data")
sdf_parquet = spark.read.format("parquet").load(path_parquet)
sdf_parquet.write.format("noop").mode("overwrite").save()

## 3-2 Column Filter

Results:
- Load data: 0.6 s

In [38]:
sc.setJobDescription("Parquet Column Filter")
sdf_parquet = spark.read.format("parquet").load(path_parquet)
sdf_parquet = sdf_parquet.select("id", "idstring")
sdf_parquet.write.format("noop").mode("overwrite").save()

## 3-3 Row Filter id

In [39]:
sc.setJobDescription("Parquet Row Filter id")
sdf_parquet = spark.read.format("parquet").load(path_parquet)
sdf_parquet = sdf_parquet.filter(f.col("id") < 300)
sdf_parquet.write.format("noop").mode("overwrite").save()

DataFrame[id: bigint, date: date, timestamp: timestamp, idstring: string, idfirst: string, idlast: string]

## 3-4 Row Filter idlast bad

In [ ]:
sc.setJobDescription("Parquet Column Filter")
sdf_parquet = spark.read.format("parquet").load(path_parquet)
sdf_parquet = sdf_parquet.filter(f.col("idlast") == "1")
sdf_parquet.write.format("noop").mode("overwrite").save()

## 3-5 Row filter idlast good


See also partition write video: S02E05

In [ ]:
sc.setJobDescription("Save Parquet")
path_parquet = "D:/Spark/Data/format_parquet_large_rep.parquet"
sdf.repartition(10, "idlast").write.format("parquet").mode("overwrite").save(path_parquet)

In [ ]:
sc.setJobDescription("Parquet Column Filter")
sdf_parquet = spark.read.format("parquet").load(path_parquet)
sdf_parquet = sdf_parquet.filter(f.col("idlast") == "1")
sdf_parquet.write.format("noop").mode("overwrite").save()

# 4 - Avro

## 4-1 Initial state AVRO
- Write time: 1.9 s
- Load time: 1.6 s
- Data size: 69.2 MB

How it works: https://spark.apache.org/docs/latest/sql-data-sources-avro.html

In [40]:
sc.setJobDescription("Save Avro")
path_avro = "D:/Spark/Data/format_avro_large.avro"
sdf.write.format("avro").mode("overwrite").save(path_avro)

In [41]:
#set maxPartitions MB for a fair comparison of 8 partitions also during read 
maxPartitionsMB = 10
maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")

In [42]:
sc.setJobDescription("Load Avro")
sdf_avro = spark.read.format("avro").load(path_avro)
sdf_avro.write.format("noop").mode("overwrite").save()

## 4-2 Column Filter

Results:
- Load data: 1.6 s

In [43]:
sc.setJobDescription("Parquet Column Filter")
sdf_avro = spark.read.format("avro").load(path_avro)
sdf_avro = sdf_avro.select("id", "idstring")
sdf_avro.write.format("noop").mode("overwrite").save()

## 4-3 Row Filter

In [ ]:
sc.setJobDescription("Parquet Column Filter")
sdf_avro = spark.read.format("avro").load(path_avro)
sdf_avro = sdf_avro.select("id", "idstring")
sdf_avro.write.format("noop").mode("overwrite").save()